In [1]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import os
import sys
# sys.path.append("..")

In [2]:
PROCESS_DATA = False

if PROCESS_DATA:
    is_first = True
    for i in range(1, 7):
        products_folder = f"sbermarket_tab_2_{i}"
        for file in os.listdir(products_folder):
            products_file = os.path.join(products_folder, file)
            if is_first:
                products = pd.read_csv(products_file, usecols=["user_id", "product_id", "quantity"])
                is_first = False
            else:
                new_products = pd.read_csv(products_file, usecols=["user_id", "product_id", "quantity"])
                products = pd.concat([products, new_products])

    products.rename(columns={"quantity": "purchase_count",
                             "user_id": "customerId",
                             "product_id": "productId"}, inplace=True)

    data = products.groupby(["customerId", "productId"]).agg({"purchase_count": "sum"})
    del products
    data.reset_index(inplace=True)
    data.to_csv("data.csv")
    del data

else:
    data = pd.read_csv("data.csv", index_col=0)
    print(f"Size  of the dataframe is {sys.getsizeof(data) / 1024**3:.1f} GB")
    print(f"Shape of the dataframe is {data.shape}")
    data.head()

Size  of the dataframe is 1409.7 MB
Shape of the dataframe is (46194236, 4)


In [ ]:
data.head()

In [3]:
sample_submission = "sample_submission.csv"
customers = pd.read_csv(sample_submission, usecols=["Id"])
users_to_recommend = customers.Id.tolist()

user_id = 'customerId'
item_id = 'productId'
n_rec = 50 # number of items to recommend

In [4]:
train_data = tc.SFrame(data)
del data

In [5]:
def fit_predict(train_data, name, user_id, item_id, target, users_to_recommend, n_rec):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec).to_dataframe()
    return recom

In [6]:
name = 'popularity'
target = 'purchase_count'
popularity = fit_predict(train_data, name, user_id, item_id, target, users_to_recommend, n_rec)
popularity.to_csv("popularity.csv")
del popularity

Warning: Ignoring columns Unnamed: 0;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 46194236 observations with 657502 users and 95002 items.

Data prepared in: 14.7739s

...

recommendations finished on 103000/107068 queries. users per second: 2040.59

recommendations finished on 104000/107068 queries. users per second: 2039

recommendations finished on 105000/107068 queries. users per second: 2037.54

recommendations finished on 106000/107068 queries. users per second: 2036.86

recommendations finished on 107000/107068 queries. users per second: 2023.79

In [9]:
name = 'cosine'
target = 'purchase_count'
cos = fit_predict(train_data, name, user_id, item_id, target, users_to_recommend, n_rec)
cos.to_csv("cos.csv")
del cos

Warning: Ignoring columns Unnamed: 0;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 46194236 observations with 657502 users and 95002 items.

Data prepared in: 15.8996s

...

recommendations finished on 103000/107068 queries. users per second: 1699.19

recommendations finished on 104000/107068 queries. users per second: 1698.63

recommendations finished on 105000/107068 queries. users per second: 1696.31

recommendations finished on 106000/107068 queries. users per second: 1695.06

recommendations finished on 107000/107068 queries. users per second: 1674.46

In [31]:
def create_output(df_rec, name):
    df_rec['Predicted'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: ' '.join(x.astype(str)))
    df_rec.rename(columns={"customerId": "Id"}, inplace=True)
    df_output = df_rec[['Id', 'Predicted']].drop_duplicates() \
        .sort_values('Id').set_index('Id')
    df_output.to_csv('submission_' + name + '.csv')
    return df_output

In [32]:
name = 'popularity'
popularity = pd.read_csv(name + ".csv", index_col=0)
popularity.head()
popularity_submission = create_output(popularity, name)

In [33]:
popularity_submission.head()

,Predicted
Id,
51,13344338 5061861 5481337 13344339 96412 746894...
65,13344338 5061861 5481337 13344339 96412 746894...
187,13344338 5061861 5481337 13344339 96412 746894...
400,13344338 5061861 5481337 13344339 96412 746894...
576,13344338 5061861 5481337 13344339 96412 746894...


In [19]:
sample_sub = pd.read_csv(sample_submission)
print(sample_sub.shape)
sample_sub.head()

(107068, 2)


,Id,Predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...
3,1132,8613934 3599065 575614 4821474 11609320 699047...
4,1578,289404 4328240 3129788 9390886 11166655 173077...


In [34]:
popularity_submission = pd.read_csv("submission_popularity.csv")
print(popularity_submission.shape)
popularity_submission.head()

(107068, 2)


,Id,Predicted
0,51,13344338 5061861 5481337 13344339 96412 746894...
1,65,13344338 5061861 5481337 13344339 96412 746894...
2,187,13344338 5061861 5481337 13344339 96412 746894...
3,400,13344338 5061861 5481337 13344339 96412 746894...
4,576,13344338 5061861 5481337 13344339 96412 746894...


In [ ]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
data_dummy = data.copy()
data_dummy['purchase_dummy'] = 1

In [ ]:
data_norm = data.copy()
data_norm["max_count_prod"] = data_norm.groupby('productId')["purchase_count"].transform('max')
data_norm["scaled_purchase_freq"] = data_norm["purchase_count"] / data_norm["max_count_prod"]
data_norm.drop(columns=['max_count_prod', 'purchase_count'], inplace=True)

In [ ]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [ ]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [ ]:
df = pd.DataFrame({'cus': [1, 1, 2, 2, 3],
                   'prod': [1, 2, 1, 2, 1],
                   'cnt': [1, 5, 3, 2, 42]})
df['max_cnt'] = df.groupby('prod')["cnt"].transform('max')
df

In [ ]:
USR = 51

In [ ]:
orders_file = "tab_1_orders.csv"
orders = pd.read_csv(orders_file, usecols=["order_id", "user_id"]) #, index_col="order_id", parse_dates=["order_completed_date"]
orders1 = orders.iloc[orders.user_id == USR].order_id.values().tolist()
# orders.head()

In [ ]:
# show_head = True
orders2 = []
for i in range(1, 7):
    products_folder = f"kaggle_tab_2_{i}"
    for file in os.listdir(products_folder):
        products_file = os.path.join(products_folder, file)
        products = pd.read_csv(products_file, usecols=["order_id", "user_id"])
        orders2 += set(products.iloc[orders.user_id == USR].order_id.values.tolist())
#         if show_head:
#             orders.head()
#             show_head = False
#         del products

In [ ]:
print(set(orders1).difference(orders2))
print(set(orders2).difference(orders1))